In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

In [ ]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [ ]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [ ]:
ROOM_CONTENTS = {
    'cemetery of ash'     : {'coiled sword'},
    'firelink shrine'     : {'estus flask'},
    'locked tower'        : {'uchiagatana'},
    'high wall of lothric': {'cell key', 'lothric banner'},
    'farron keep'         : {'soul of the blood of the wolf'},
}

ITEM_WEIGHT = {
                 'coiled sword' : 5,
                  'estus flask' : 4,
                  'uchiagatana' : 4,
                     'cell key' : 2,
'soul of the blood of the wolf' : 5,
               'lothric banner' : 5
}

DOORS = [
    Door( 'cemetery of ash', 'firelink shrine' ),
    Door( 'firelink shrine', 'high wall of lothric' ),
    Door( 'firelink shrine', 'locked tower', doorkey='cell key', locked=True ),
    Door( 'high wall of lothric', 'farron keep', doorkey='lothric banner', locked=True ),
]


In [ ]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough
        for i in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
            elif door.locked==True and robot_location in door.goes_between:
                if door.doorkey in state.robot.carried_items:
                    actions.append( ("move to", door.other_loc[robot_location]) )

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

In [ ]:
def cost(path, state):
    """
    This is an optional method that you only need to define if you are using
    a cost based algorithm such as "uniform cost" or "A*". It should return
    the cost of reaching a given state via a given path.
    If this is not re-defined, it will is assumed that each action costs one unit
    of effort to perform, so it returns the length of the path.
    """
    return len(path)

In [ ]:
rob = Robot('cemetery of ash', [], 10 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {"firelink shrine":{"soul of the blood of the wolf", "coiled sword"},
                        "farron keep":{"uchiagatana", "estus flask"}}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

In [ ]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

In [ ]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

In [ ]:
def goal_items_heuristic(state):
    items_needed = 0
    for room, contents in goal_item_locations.items():
        for i in contents:
            if not i in state.room_contents[room]:
                items_needed += 1
    return items_needed

def minimize_visits_heuristic(state):
    visits_needed = 0
    for room, contents in goal_item_locations.items():
        items_needed = contents - state.room_contents[room]
        visits_needed += len(items_needed)
    return visits_needed

def keys_carried_heuristic(state):
    keys_needed = 0
    for door in state.doors:
        if door.locked and not door.doorkey in state.robot.carried_items:
            keys_needed += 1
    return keys_needed

In [ ]:
import pandas as pd

# Initialize lists to store results
heuristics = [None, keys_carried_heuristic, minimize_visits_heuristic, goal_items_heuristic]
costs = [None, cost]
average_runtimes = []
average_path_lengths = []
average_nodes_generated = []
method_heuristic_pairs = []

# Initialize dictionaries to store results
method_runtimes = {}
method_path_lengths = {}
method_nodes_generated = {}
method_exit_conditions = {}

# Run 10 iterations
for _ in range(1):
    # Run 'BF/FIFO' method with no heuristic
    method_key = ('BF/FIFO', 'None')
    result = search(RW_PROBLEM_1, 'BF/FIFO', 200000, loop_check=True, randomise=True, heuristic=None, return_info=True)
    method_runtimes.setdefault(method_key, []).append(result['search_stats']['time_taken'])
    path_length = result['result']['path_length']
    if path_length is not None:
        method_path_lengths.setdefault(method_key, []).append(path_length)
    method_nodes_generated.setdefault(method_key, []).append(result['search_stats']['nodes_generated'])
    method_exit_conditions.setdefault(method_key, []).append(result['result']['termination_condition'])

    # Run 'DF/LIFO' method with each heuristic
    for cost in costs:
        for heuristic in heuristics:
            method_key = ('DF/LIFO', heuristic.__name__ if heuristic else 'None', cost.__name__ if cost else 'None')
            result = search(RW_PROBLEM_1, 'DF/LIFO', 200000, loop_check=True, randomise=True, cost=cost, heuristic=heuristic, return_info=True)
            method_runtimes.setdefault(method_key, []).append(result['search_stats']['time_taken'])
            path_length = result['result']['path_length']
            if path_length is not None:
                method_path_lengths.setdefault(method_key, []).append(path_length)
            method_nodes_generated.setdefault(method_key, []).append(result['search_stats']['nodes_generated'])
            method_exit_conditions.setdefault(method_key, []).append(result['result']['termination_condition'])

# Calculate averages and determine exit conditions
average_runtimes = {k: sum(v) / len(v) for k, v in method_runtimes.items()}
average_path_lengths = {k: sum(v) / len(v) if v else None for k, v in method_path_lengths.items()}
average_nodes_generated = {k: sum(v) / len(v) for k, v in method_nodes_generated.items()}
exit_conditions = {k: 'GOAL_STATE_FOUND' if 'NODE_LIMIT_EXCEEDED' not in v else 'NODE_LIMIT_EXCEEDED' for k, v in method_exit_conditions.items()}

# Create a DataFrame to store the results
df = pd.DataFrame({
    'Method and Heuristic': list(average_runtimes.keys()),
    'Average Runtime (s)': list(average_runtimes.values()),
    'Average Path Length': list(average_path_lengths.values()),
    'Average Nodes Generated' : list(average_nodes_generated.values()),
    'Exit Condition' : list(exit_conditions.values())
})

# Display the DataFrame
print(df)